# Tutorial on how to use mescal
*mescal* is a [Brightway](https://docs.brightway.dev/en/latest/)-powered Python package that helps you to integrate Life-Cycle Assessment (LCA) in your energy system model

## Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import the required libraries
from mescal import *
import pandas as pd
import bw2data as bd

In [3]:
ecoinvent_version = '3.9.1' # choose the ecoinvent version you wish to use
esm_location = 'CH' # choose the version of energyscope for which you want to generate metrics
esm_year = 2020 # choose the year of the energyscope snapshot model or transition pathway time-step
spatialized_database = False # set to True if you want to use your spatialized version of ecoinvent
regionalize_foregrounds = True # set to True if you want to regionalize the foreground inventories
premise_iam = 'image' # choose the IAM to which the premise database is linked
premise_ssp_rcp = 'SSP2-Base' # choose the SSP/RCP scenario to which the premise database is linked

In [4]:
# Set the name of your main LCI database (e.g., ecoinvent or premise database) here:
name_premise_db = f"ecoinvent_cutoff_{ecoinvent_version}_{premise_iam}_{premise_ssp_rcp}_{esm_year}+truck_carculator"
if spatialized_database:
    name_premise_db += ' regionalized'
    name_spatialized_biosphere_db = 'biosphere3_spatialized_flows'
else:
    name_spatialized_biosphere_db = None

In [5]:
# Set the name of the new database with the ESM results
esm_db_name = f'EnergyScope_{esm_location}_{esm_year}'

In [6]:
# Main version of ecoinvent (without .1 if any)
ecoinvent_main_version = '.'.join(ecoinvent_version.split('.')[:2])

In [7]:
# Set the list of LCIA methods for which you want indicators (they must be registered in your brightway project)
if spatialized_database:
    lcia_methods=[
        'IMPACT World+ Midpoint 2.1_regionalized', 
        'IMPACT World+ Damage 2.1_regionalized',
    ]
else:
    lcia_methods=[
        f'IMPACT World+ Midpoint 2.1 for ecoinvent v{ecoinvent_main_version}', 
        f'IMPACT World+ Damage 2.1 for ecoinvent v{ecoinvent_main_version}', 
        f'IMPACT World+ Footprint 2.1 for ecoinvent v{ecoinvent_main_version}', 
    ]

**Note**: you can download IMPACT World+ methods in your brightway project following this [notebook](https://github.com/matthieu-str/mescal/blob/master/dev/download_impact_world_plus.ipynb). Regionalized versions of ecoinvent and IMPACT World+ methods can be downloaded from this [repository](https://github.com/matthieu-str/Regiopremise).

In [8]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ecoinvent_version}A') # put the name of your brightway project here

## Load databases

In [9]:
premise_db = Database(name_premise_db)

Getting activity data


100%|██████████| 33941/33941 [00:01<00:00, 31131.41it/s]


Adding exchange data to activities


100%|██████████| 987429/987429 [01:24<00:00, 11745.61it/s]


Filling out exchange data


100%|██████████| 33941/33941 [00:06<00:00, 5390.62it/s] 


Loaded ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+truck_carculator from brightway!


In [10]:
if regionalize_foregrounds & spatialized_database:
    spatialized_biosphere_db = Database(name_spatialized_biosphere_db)
else:
    spatialized_biosphere_db = None

## Your data
For mescal to understand the structure of your energy system model, you need to provide it with a set of dataframes.

### Mandatory dataframes

In [11]:
mapping = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}.csv')
unit_conversion = pd.read_excel(f'../dev/energyscope_data/{esm_location}/unit_conversion_{ecoinvent_version}.xlsx')
mapping_esm_flows_to_CPC = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_esm_flows_to_CPC.csv')
model = pd.read_csv(f'../dev/energyscope_data/{esm_location}/model.csv')

A mapping between the energy technologies and resources of the energy system model, and Life-Cycle Inventories datasets (LCI) from an LCI database (e.g., ecoinvent). The mapping should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', 'Resource' or 'Flow')  
- `Product`: the name of the product of the energy technology or resource in the LCI database
- `Activity`: the name of the activity of the energy technology or resource in the LCI database
- `Location`: the name of the location of the energy technology or resource in the LCI database
- `Unit`: (optional) the physical unit of the energy technology or resource in the LCI database
- `Database`: the name of the database in your brightway project

If you wish to change the version of ecoinvent used in your mapping file, you can follow this [notebook](https://github.com/matthieu-str/mescal/blob/master/dev/mapping_ecoinvent_version.ipynb). 

In [14]:
mapping.head()

,Name,Type,Product,Activity,Location,Database
0,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
1,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
2,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
3,ALKALINE_ELECTROLYSIS_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
4,ALKALINE_ELECTROLYSIS_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020


A set of unit conversion factors between the energy system model and the LCI database. The conversion factors should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy system model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', 'Resource', or 'Other'). Other types can be added to fit your needs. The 'Other' category is meant for the unit conversion that are not specific to a technology or resource, but rather a generic type of product, e.g., conversion from kilogram to kilowatt hour for natural gas.
- `Value`: the numerical value of the conversion factor that will multiply the impact scores. It actually denotes the conversion from Impact / LCA unit to Impact / ESM unit. 
- `ESM`: the unit of the energy technology or resource in the ESM
- `LCA`: the target unit of the energy technology or resource in the LCA database
- `Assumptions & Sources` (optional): additional information on the conversion factor.

The `LCA` and `ESM` columns should respect the ecoinvent naming convention. You may use the `ecoinvent_unit_convention` function to convert the unit naming convention you've used to the one of ecoinvent.

In [15]:
unit_conversion.head()

,Name,Type,Value,LCA,ESM,Assumptions & Sources
0,ACETIC_ACID,Resource,0.247423,kilogram,kilowatt hour,acetic acid heat of combustion (14.55 MJ/kg): ...
1,ACETONE,Resource,0.121655,kilogram,kilowatt hour,Acetone LHV (8.22 kWh/kg): \cite{engineeringto...
2,ALKALINE_ELECTROLYSIS,Construction,0.001556,unit,kilowatt,Electricity consumption (51.8 kWh/kg H2): \cit...
3,ALKALINE_ELECTROLYSIS,Operation,0.030030,kilogram,kilowatt hour,Electricity consumption (51.8 kWh/kg H2): \cit...
4,ALKALINE_ELECTROLYSIS_PLANT,Construction,1.000000,unit,unit,NaN


A mapping between the energy system model flows and CPC categories. The mapping should be provided in a dataframe with the following columns:
- `Flow`: the name of the flow in the ESM
- `Description`: (optional) the description of the product
- `CPC`: the list of names of corresponding CPC categories

In [13]:
mapping_esm_flows_to_CPC.head()

,Flow,Description,CPC
0,BENZENE,Benzene,"['33100: Coke and semi-coke of coal, of lignit..."
1,BIO_DIESEL,Bio-diesel,"['35491: Biodiesel', '33370: Fuel oils n.e.c.'..."
2,CO2_A,Carbon dioxide (concentrated emissions),['34210b: Carbon dioxide and monoxide']
3,CO2_C,Carbon dioxide (captured),['34210b: Carbon dioxide and monoxide']
4,CO2_CS,Carbon dioxide (captured & stored),['34210b: Carbon dioxide and monoxide']


The input and output flows of energy technologies. For a given technology, the inputs and outputs should be given with the same unit. Also, the main output flow should have 1 as an amount, i.e., all other flows as scaled with respect to the main output. It should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy system model
- `Flow`: the name of the input or output flow
- `Amount`: the numerical value of the flow (negative if input, positive if output)  

In [14]:
model.head()

,Name,Flow,Amount
0,ACETIC_ACID,ACETIC_ACID,1.000
1,ACETONE,ACETONE,1.000
2,AFC,ELECTRICITY_LV,2.915
3,AFC,H2_HP,-4.255
4,AFC,HEAT_LOW_T_DECEN,1.000


### Optional dataframes

In [12]:
technology_compositions = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_compositions.csv')
technology_specifics = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_specifics.csv') 
lifetime = pd.read_csv(f'../dev/energyscope_data/{esm_location}/lifetime.csv')
efficiency = pd.read_csv(f'../dev/energyscope_data/{esm_location}/efficiency.csv')
mapping_product_to_CPC = pd.read_csv('../mescal/data/mapping_product_to_CPC.csv')
impact_abbrev = pd.read_csv('../dev/lcia/impact_abbrev.csv')
technologies_to_remove_from_layers = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technologies_to_remove_from_layers.csv')
new_end_use_types = pd.read_csv(f'../dev/energyscope_data/{esm_location}/new_end_use_types.csv')
premise_changes = pd.read_csv('../dev/data/premise_change_report.csv')
results_from_esm = pd.read_csv(f'../dev/energyscope_data/{esm_location}/results_ES.csv')

A set of composition of technologies, i.e., if one technology or resource in the energy model should be represented by a combination of LCI datasets. The composition should be provided in a dataframe with the following columns:
- `Name`: the name of the main energy technology or resource in the energy model 
- `Components`: the list of names of subcomponents

In [16]:
technology_compositions.head()

,Name,Components
0,ALKALINE_ELECTROLYSIS,"['ALKALINE_ELECTROLYSIS_STACK', 'ALKALINE_ELEC..."
1,AN_DIG_SI,"['AN_DIG_SI_PLANT', 'AN_DIG_SI_COGEN']"
2,ATR,"['ATR_PLANT', 'ATR_TANK']"
3,ATR_CCS,"['ATR_CCS_PLANT', 'ATR_CCS_TANK']"
4,BIOGAS_ATR,"['BIOGAS_ATR_PLANT', 'BIOGAS_ATR_TANK']"


A set of technologies with specific requirements. For instance, this stands for energy technologies without a construction phase, mobility technologies (if mismatch fuel in the LCI dataset), bio-processes (if mismatch fuel in the LCI dataset), etc. The requirements should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `Specifics`: the type of requirement. Can be '**Background search**' (i.e., double-counting removal is run n levels in the background, n being defined in Amount), '**Decommissioning**' (i.e., the technology has a decommissioning LCI dataset outside its infrastructure LCI dataset), '**Mobility**' (i.e., EUD types for which associated technologies are characterized as a mobility mean, to further add a FUEL layer), '**No background search**' (i.e., double-counting removal is not applied), '**No construction**' (i.e., the technology has no infrastructure LCI dataset), '**Process**' (i.e., the technology is characterized as an industrial bio-process, to further add a PROCESS_FUEL layer), '**DAC**' (for *premise* DAC technologies, to change the biogenic carbon flow into a fossil one), '**Biofuel**' (i.e., adapt direct emissions to the biofuel input: partially change fossil carbon emissions into biogenic ones), '**Import/Export**' (these activities will keep their original locations and will not be regionalized), '**Carbon flows**' (the direct carbon emissions of these activities will be multiplied by a factor), '**Add CC**' (add a carbon capture process to a technology, and modifies its direct carbon dioxide emissions according to the capture efficiency). 
- `Amount`: the numerical value of the requirement (if relevant)

In [17]:
technology_specifics.head()

,Name,Specifics,Amount,Comment
0,CO2_TO_DIESEL,Background search,2,NaN
1,CO2_TO_JETFUELS,Background search,0,NaN
2,CROPS_TO_JETFUELS,Background search,4,NaN
3,FT,Background search,3,NaN
4,GASIFICATION_SNG,Background search,2,NaN


Energy technologies lifetimes in the ESM and the LCI database. For composition of technologies (if any), the main technology should not have a LCA lifetime (no LCI dataset is associated to it), while the sub-components should not have an ESM lifetime (they do not have a proper technology in the ESM). The lifetimes should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy system model
- `ESM`: the numerical value of the lifetime in the energy system model (if relevant)
- `LCA`: the numerical value of the lifetime in the LCI database (if relevant)
- `Comment`: (optional) additional information the `Amount` computation (if relevant)

In [18]:
lifetime.head()

,Name,ESM,LCA
0,AFC,35.0,2.686006
1,ALKALINE_ELECTROLYSIS,10.0,NaN
2,ALKALINE_ELECTROLYSIS_PLANT,NaN,20.000000
3,ALKALINE_ELECTROLYSIS_PLANT_DECOM,NaN,20.000000
4,ALKALINE_ELECTROLYSIS_STACK,NaN,7.500000


If you want to account for the possible efficiency differences between the ESM and the LCI datasets, you can provide a set of (technologies, flow) couples for which efficiency differences will be corrected by the scaling the elementary flows. Taking the example of the diesel car, the couple should be something like ('CAR_DIESEL', 'DIESEL'), the flow being the fuel of the technology responsible for direct emissions. Relevant biosphere flows will be the ratio between the LCA and ESM efficiencies. The (technology, flow) couples should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the ESM
- `Flow`: the name of the flow in the ESM
- `Comment`: (optional) additional information on the set of `Flows`

In [19]:
efficiency.head()

,Name,Flow,Comment
0,AN_DIG,['WET_BIOMASS'],NaN
1,AN_DIG_SI,['WET_BIOMASS'],NaN
2,ATR,['NG_HP'],NaN
3,ATR_CCS,['NG_HP'],NaN
4,BIOGAS_ATR,['WOOD'],NaN


In case you have a LCI database (partially) without CPC categories (which are necessary for the double-counting check), you can provide a mapping between the products and activities in the LCI database and the CPC categories. The mapping should be provided in a dataframe with the following columns:
- `Name`: the full or partial name of the product or activity in the LCI database
- `CPC`: the number and name of the corresponding CPC category
- `Search type`: can be 'equals' if the `Name` entry is an exactly the name to look for, or 'contains' if it is contained in the full name
- `Where`: can be 'Product' or 'Activity', whether the `Name` entry is meant for products or activities

In [20]:
mapping_product_to_CPC.head()

,Name,CPC,Search type,Where
0,amine-based silica,"35310: Organic surface active agents, except soap",equals,Product
1,biodiesel,35491: Biodiesel,contains,Product
2,biogas,"17200: Coal gas, water gas, producer gas and s...",equals,Product
3,"biomass, used as fuel",31230: Wood in chips or particles,equals,Product
4,"biomethane, from biogas upgrading, using amine...","12020: Natural gas, liquefied or in the gaseou...",equals,Product


An abbreviation scheme for the impact categories you aim to work with, to ease the readability in the ESM. The one of IMPACT World+ is available in this [csv file](https://github.com/matthieu-str/mescal/blob/master/dev/lcia/impact_abbrev.csv). The abbreviations should be provided in a dataframe with the following columns:
- `Impact_category`: the name of the impact category, expressed as a tuple following brightway convention
- `Unit`: (optional) the unit of the impact category
- `Abbrev`: the abbreviation of the impact category
- `AoP`: the area of protection of the impact category. The normalization of indicators will be performed based of AoPs. If you do not have AoPs (e.g., midpoint-level indicators), set as many AoPs as you have impact categories.

In [21]:
impact_abbrev.head()

,Impact_category,Unit,Abbrev,AoP
0,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQL,EQ
1,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQS,EQ
2,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQLB,EQ
3,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQSB,EQ
4,"('IMPACT World+ Damage 2.0.1', 'Human health',...",DALY,CCHHL,HH


In case you want to remove some energy technologies from energy layers in the results LCI datasets to be added to the LCI database, you can provide a list of technologies to remove. The list should be provided in a dataframe with the following columns:
- `Layers`: name of the layer(s). A layer is basically an energy vector, which is an output for some energy technologies and an input for some others. 
- `Technologies`: the name of the energy technology to remove from the layer(s)
- `Comment`: (optional) a comment on the removal

In [22]:
technologies_to_remove_from_layers.head()

,Layers,Technologies,Comment
0,"['ELECTRICITY_EHV', 'ELECTRICITY_HV']","['TRAFO_HE','TRAFO_EH']",The high and extra high voltage electricity ar...
1,['ELECTRICITY_LV'],['STO_ELEC'],The storage technologies should be removed (pr...
2,"['NG_HP', 'NG_EHP']","['NG_EXP_EH', 'NG_EXP_EH_COGEN', 'NG_COMP_HE',...",The high and extra high pressure natural gas a...
3,"['H2_LP', 'H2_MP', 'H2_HP', 'H2_EHP']","['H2_COMP_HE', 'H2_COMP_MH', 'H2_COMP_LM', 'H2...",All pressure levels for hydrogen are merged in...
4,"['HEAT_HIGH_T', 'HEAT_LOW_T_DHN']",['HT_LT'],The high and low heat production at the DHN le...


If you want to reformat your end-use types (e.g., output layer) in order to better fit the LCI database, you can provide a list of new end-use types. The list should be provided in a dataframe with the following columns:
- `Name`: name of technologies for which the end-use type should be changed
- `Search type`: whether the `Name` entry is an exactly the name to look for ('equals'), is contained in the full name ('contains'), or is the beginning of the full name ('startswith')
- `Old`: the current end-use type
- `New`: the new end-use type

In [23]:
new_end_use_types.head()

,Name,Search type,Old,New
0,BUS,startswith,MOB_PUBLIC,MOB_PUBLIC_BUS
1,SCHOOLBUS,startswith,MOB_PUBLIC,MOB_PUBLIC_SCHOOLBUS
2,COACH,startswith,MOB_PUBLIC,MOB_PUBLIC_COACH
3,TRAIN,startswith,MOB_PUBLIC,MOB_PUBLIC_TRAIN
4,CAR,startswith,MOB_PRIVATE,MOB_PRIVATE_CAR


If you are facing changes in names or reference products due to a new version of premise, and do not want to modify your mapping file, you should provide the dataframe with the following columns:
- `Product - old`: old reference product of the activity
- `Activity - old`: old name of the activity
- `Location - old`: old location of the activity
- `Product - new`: new reference product of the activity
- `Activity - new`: new name of the activity
- `Location - new`: new location of the activity

In [24]:
premise_changes.head()

,Product - old,Activity - old,Location - old,Product - new,Activity - new,Location - new
0,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming of...",RER,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming",RER
1,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming of...",RER,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, w...",RER
2,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming, f...",CH,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, f...",RER
3,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming, f...",CH,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, f...",RER
4,"hydrogen, gaseous, 26 bar","hydrogen production, steam methane reforming, ...",CH,"hydrogen, gaseous, low pressure","hydrogen production, steam methane reforming, ...",RER


If you want to inject the results of your ESM back in the LCI database, you should provide the results in a dataframe with the following columns:
- `Name`: the name of the energy technology in the ESM
- `Production`: the annual production value of the energy technology in the ESM. All values should be provided with the same unit.

In [25]:
results_from_esm.head()

,Name,Production
0,PV,0.000000
1,TRAFO_ML,72698.861206
2,AN_DIG_SI,0.000000
3,GEOTHERMAL,0.000000
4,HYDRO_GAS_CHP,0.000000


### Optional geographical data

In [13]:
# sufficient match within ecoinvent
if esm_location == 'CA-QC':
    accepted_locations = ['CA-QC', 'CA']
elif esm_location == 'CH':
    accepted_locations = ['CH']
else:
    accepted_locations = ['GLO', 'RoW']

In [14]:
# Define the user-defined ranking
if esm_location == 'CA-QC':
    my_ranking = [
        'CA-QC', # Quebec
        'CA', # Canada
        'CAN', # Canada in IMAGE
        'CAZ', # Canada - Australia - New Zealand in REMIND
        'RNA', # North America
        'US', # United States
        'USA', # United States in REMIND and IMAGE
        'GLO', # Global average 
        'RoW', # Rest of the world
    ]
elif esm_location == 'CH':
    my_ranking = [
        'CH',
        'NEU',
        'EUR',
        'WEU',
        'RER', 
        'IAI Area, EU27 & EFTA',
        'GLO',
        'RoW'
    ]
else:
    my_ranking = [
        'GLO',
        'RoW',
    ]

## Create a new database with additional CPC categories (optional)
In case you are working with a LCI database without CPC categories, you can create a new database with the CPC categories. The function `create_new_database_with_CPC_categories` takes as input the database with missing CPC categories, the name of the new database, and a mapping between the products and activities in the LCI database and the CPC categories. It creates a new database with the CPC categories. This step can take a few minutes depending on the size of the database.

In [15]:
# If necessary, add missing CPC categories to the database
premise_db.add_CPC_categories()

## Relink the mapping dataframe with another database (optional)
In this example, we relink the current mapping, which is based on ecoinvent and some additional inventories, with a [premise](https://premise.readthedocs.io/en/latest/introduction.html) database. The function `create_complementary_database` takes as input the mapping dataframe, the database to link to, and the name of a newly created complementary database, in case some LCI datasets are missing in the LCI database we relink to. It returns a new mapping dataframe with the location column updated, and create the complementary database in the brightway project.

In [16]:
name_premise_comp_db = name_premise_db + f'_comp_{esm_location}'

In [17]:
mapping_linked_to_premise = premise_db.create_complementary_database(
    df_mapping=mapping,
    complement_db_name=name_premise_comp_db,
    premise_changes=premise_changes,
)

Getting activity data


100%|██████████| 28551/28551 [00:01<00:00, 16665.68it/s]


Adding exchange data to activities


100%|██████████| 838277/838277 [01:16<00:00, 10923.90it/s]


Filling out exchange data


100%|██████████| 28551/28551 [00:05<00:00, 4820.75it/s] 


Loaded ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020 from brightway!
The complementary database did not have to be created


In [18]:
if name_premise_db+f'_comp_{esm_location}' in bd.databases: # if the complementary database exists, load it and update it with CPC categories
    premise_comp_db = Database(db_names=name_premise_db) # complementary database
    premise_comp_db.add_CPC_categories() # if necessary, add CPC categories
    main_db = premise_db + premise_comp_db # concatenate the two databases in one database object (without writing it)
else:
    main_db = premise_db

In [19]:
mapping_linked_to_premise.head()

,Name,Type,Product,Activity,Location,Database
0,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
1,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
2,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
3,ALKALINE_ELECTROLYSIS_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
4,ALKALINE_ELECTROLYSIS_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...


## Initialize the ESM database

In [20]:
# To skip the previous steps
# mapping_linked_to_premise = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_linked.csv')

In [21]:
esm = ESM(
    # Mandatory inputs
    mapping=mapping_linked_to_premise,
    unit_conversion=unit_conversion,
    model=model,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    main_database=main_db,
    esm_db_name=esm_db_name,
    
    # Optional inputs
    technology_compositions=technology_compositions,
    tech_specifics=technology_specifics,
    lifetime=lifetime,
    efficiency=efficiency,
    regionalize_foregrounds=regionalize_foregrounds,
    accepted_locations=accepted_locations,
    locations_ranking=my_ranking,
    esm_location=esm_location,
    spatialized_biosphere_db=spatialized_biosphere_db,
    results_path_file=f'results/energyscope_{esm_location}/{esm_year}/',
)

In [22]:
esm.check_inputs()

List of technologies or resources that are in the model file but not in the mapping file. Their impact scores will be set to the default value.

--> ['CO2_CS', 'CO2_E', 'DEC_RENOVATION', 'DHN_RENOVATION', 'DIESEL_S', 'ELEC_S', 'GASOLINE_S', 'H2_S', 'HT_LT', 'LT_DEC_WH', 'LT_DHN_WH', 'NG_S', 'RES_GEO', 'RES_HYDRO', 'RES_SOLAR', 'RES_WIND', 'STO_CO2', 'STO_DIE', 'STO_ELEC', 'STO_GASO', 'STO_H2', 'STO_NG']

List of technologies or resources that are in the mapping file but not in the model file (this will not be a problem in the workflow).

--> ['BATTERY', 'CAR_BEV_LOWRANGE', 'CAR_DME_D10_LOCAL', 'CAR_DME_D10_LONGD', 'CEMENT_PROD', 'CEMENT_PROD_HP', 'DEC_TH_STORAGE', 'DHN_COGEN_WASTE_CC', 'DHN_TH_STORAGE', 'EHP_H2_GRID', 'EHP_NG_GRID', 'EHV_GRID', 'HP_H2_GRID', 'HP_NG_GRID', 'HV_GRID', 'IND_BOILER_WASTE_CC', 'IND_COGEN_WASTE_CC', 'LP_H2_GRID', 'LP_NG_GRID', 'LV_GRID', 'MP_H2_GRID', 'MP_NG_GRID', 'MV_GRID']

List of technologies that are in the tech_specifics file but not in the mapping fi

## Add or replace the location column based on a user-defined ranking (optional)
Based on a user-defined ranking, the location column of the mapping dataframe can be updated. The function `change_location_mapping_file` takes as input the mapping dataframe, the user-defined ranking, and the base database. It returns the mapping dataframe with the location column updated.

In [23]:
# Update mapping dataframe with better locations
esm.change_location_mapping_file()

In [24]:
esm.mapping.head() 

,Name,Type,Product,Activity,Location,Database
0,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
1,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
2,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
3,ALKALINE_ELECTROLYSIS_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...
4,ALKALINE_ELECTROLYSIS_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+tr...


In [25]:
# Save the relinked mapping file for later use
esm.mapping.to_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_linked.csv', index=False)

## Create ESM database after double-counting removal, efficiency harmonization, and lifetime harmonization

In [35]:
# Load the relinked mapping file to skip the previous steps
# mapping_linked_to_premise = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_linked.csv')
# esm.mapping = mapping_linked_to_premise

In [36]:
esm.create_esm_database()

### Starting to add construction and resource activities database ###
No location found in your ranking for iridium - platinum group metal, extraction and refinery operations
--> Have to keep the initial location: ZA
No location found in your ranking for natural gas, liquefied - natural gas production, liquefied
--> Have to keep the initial location: QA
### Construction and resource activities added to the database ###
--> Time: 20.0 seconds
### Starting to remove double-counted flows ###
### Double-counting removal done ###
--> Time: 42.0 seconds
### Starting to correct efficiency differences ###
No flow found for type(s) ['NG_HP'] in BUS_FC_HYBRID_CH4. The efficiency of this technology cannot be adjusted.
No flow found for type(s) ['NG_HP'] in CAR_FC_CH4_LOCAL. The efficiency of this technology cannot be adjusted.
No flow found for type(s) ['NG_HP'] in CAR_FC_CH4_LONGD. The efficiency of this technology cannot be adjusted.
No flow found for type(s) ['NG_HP'] in COACH_FC_HYBRID_CH4. T

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 02/21/2025 11:10:37
  Finished: 02/21/2025 11:10:39
  Total time elapsed: 00:00:01
  CPU %: 84.80
  Memory %: 5.97
EnergyScope_CH_2020 written to Brightway!
### Database written ###
--> Time: 309.0 seconds


In [37]:
# Save the mapping file with new codes for later use
esm.mapping.to_csv(f'../examples/results/energyscope_{esm_location}/{esm_year}/mapping_{ecoinvent_version}_new_code.csv', index=False)

## Computing the LCA metrics 

In [23]:
# To skip the previous steps (if the mapping file with the new codes has already been created)
mapping_linked_to_premise_new_code = pd.read_csv(f'../examples/results/energyscope_{esm_location}/{esm_year}/mapping_{ecoinvent_version}_new_code.csv')
esm.mapping = mapping_linked_to_premise_new_code

In [26]:
activities_subject_to_double_counting = pd.read_csv(f'../examples/results/energyscope_{esm_location}/{esm_year}/activities_subject_to_double_counting.csv')

In [42]:
R_long_direct_emissions = esm.compute_impact_scores(
    methods=lcia_methods,
    assessment_type='direct emissions',
    activities_subject_to_double_counting=activities_subject_to_double_counting,
    overwrite=False,
)

Getting activity data


100%|██████████| 1385/1385 [00:00<00:00, 90497.28it/s]


Adding exchange data to activities


100%|██████████| 36560/36560 [00:02<00:00, 12975.57it/s]


Filling out exchange data


100%|██████████| 1385/1385 [00:01<00:00, 1338.36it/s]


Loaded EnergyScope_CA-QC_2050 from brightway!
EnergyScope_CA-QC_2050_direct_emissions already exists in Brightway. To overwrite it, set 'overwrite' to True or delete it manually


In [43]:
R_long_direct_emissions.to_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores_direct_emissions.csv', index=False)

In [29]:
R_long = esm.compute_impact_scores(methods=lcia_methods)

Getting activity data


100%|██████████| 846/846 [00:00<00:00, 115319.51it/s]


Adding exchange data to activities


100%|██████████| 13167/13167 [00:08<00:00, 1540.21it/s]


Filling out exchange data


100%|██████████| 846/846 [00:01<00:00, 554.35it/s]


Loaded EnergyScope_CH_2050 from brightway!


In [30]:
R_long.to_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores.csv', index=False)

In [31]:
R_long.head()

,Impact_category,New_code,Value,Name,Type
0,"(IMPACT World+ Damage 2.1 for ecoinvent v3.10,...",8ra0i43ebr1949z3dar90gf88holabg0,144.514831,AL_MAKING,Construction
1,"(IMPACT World+ Damage 2.1 for ecoinvent v3.10,...",8ra0i43ebr1949z3dar90gf88holabg0,43.722043,AL_MAKING,Construction
2,"(IMPACT World+ Damage 2.1 for ecoinvent v3.10,...",8ra0i43ebr1949z3dar90gf88holabg0,0.001948,AL_MAKING,Construction
3,"(IMPACT World+ Damage 2.1 for ecoinvent v3.10,...",8ra0i43ebr1949z3dar90gf88holabg0,0.000589,AL_MAKING,Construction
4,(IMPACT World+ Midpoint 2.1 for ecoinvent v3.1...,8ra0i43ebr1949z3dar90gf88holabg0,354.561179,AL_MAKING,Construction


## Convert the results in an AMPL format

In [22]:
# To skip the previous steps
# R_long = pd.read_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores.csv')
R_long_direct_emissions = pd.read_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores_direct_emissions.csv')

In [32]:
R_long.Value *= 1e6 # from FU / kW(h)-pkm-tkm to FU / GW(h)-Mpkm-Mtkm

In [23]:
R_long_direct_emissions.Value *= 1e6 # from FU / kW(h)-pkm-tkm to FU / GW(h)-Mpkm-Mtkm

In [33]:
# Tou can specify which specific LCIA methods or impact categories you wish to use using one of the three following options:
specific_lcia_methods = ['IMPACT World+ Footprint 2.1 for ecoinvent v3.10'] # selects the specific methods via their names
# specific_lcia_categories = ['Total ecosystem quality', 'Total human health']  # selects the specific impact categories via their names 
# specific_lcia_abbrev = ['TTHH', 'TTEQ', 'm_CCS'] # selects the specific impact categories via their abbreviations

In [34]:
# Additional information that can be added at the beginning of the AMPL .mod and .dat files
metadata = {
    'ecoinvent_version': ecoinvent_version,
    'year': esm_year,
    'spatialized': spatialized_database,
    'regionalized': regionalize_foregrounds,
    'iam': premise_iam,
    'ssp_rcp': premise_ssp_rcp,
    'lcia_methods': lcia_methods,
}

### Normalize LCA indicators and create the .dat file

In [35]:
esm.normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    lcia_methods=specific_lcia_methods,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
    metadata=metadata,
)

In [26]:
esm.normalize_lca_metrics(
    R=R_long_direct_emissions,
    mip_gap=1e-6,
    lcia_methods=lcia_methods,
    # specific_lcia_abbrev=specific_lcia_abbrev,
    assessment_type='direct emissions',
    max_per_cat=pd.read_csv(f'../examples/results/energyscope_{esm_location}/{esm_year}/res_lcia_max.csv'),
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
    metadata=metadata,
    file_name='techs_lcia_direct',
)

### Create the .mod file

In [36]:
esm.generate_mod_file_ampl(
    lcia_methods=specific_lcia_methods,
    # specific_lcia_abbrev=specific_lcia_abbrev,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
    metadata=metadata,
)

In [27]:
esm.generate_mod_file_ampl(
    lcia_methods=lcia_methods,
    # specific_lcia_abbrev=specific_lcia_abbrev,
    assessment_type='direct emissions',
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
    metadata=metadata,
    file_name='objectives_direct',
)

## Integrate the ESM results back in the LCI database

In [31]:
esm.create_new_database_with_esm_results(
    esm_results=results_from_esm,
    new_end_use_types=new_end_use_types,
    tech_to_remove_layers=technologies_to_remove_from_layers,
    write_database=True,
    remove_background_construction_flows=True,
    apply_tech_specifics_changes=True,
)

The technology DHN_RENOVATION is not in the mapping file. It cannot be considered in the result LCI dataset.
Previous EnergyScope_CH_2020_results will be overwritten!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 02/21/2025 16:32:38
  Finished: 02/21/2025 16:32:38
  Total time elapsed: 00:00:00
  CPU %: 67.20
  Memory %: 3.35
EnergyScope_CH_2020_results written to Brightway!
Getting activity data


100%|██████████| 32/32 [00:00<?, ?it/s]


Adding exchange data to activities


100%|██████████| 364/364 [00:00<00:00, 359229.80it/s]


Filling out exchange data


100%|██████████| 32/32 [00:00<00:00, 123.11it/s]


Loaded EnergyScope_CH_2020_results from brightway!


In [26]:
esm.connect_esm_results_to_database(create_new_db=True)

Getting activity data


100%|██████████| 32/32 [00:00<00:00, 15870.61it/s]


Adding exchange data to activities


100%|██████████| 364/364 [00:00<00:00, 10135.20it/s]


Filling out exchange data


100%|██████████| 32/32 [00:00<00:00, 103.90it/s]


Loaded EnergyScope_CH_2020_results from brightway!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:43


Title: Writing activities to SQLite3 database:
  Started: 02/21/2025 17:23:50
  Finished: 02/21/2025 17:25:34
  Total time elapsed: 00:01:43
  CPU %: 94.60
  Memory %: 25.74
ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020+truck_carculator_with_esm_results_for_CH written to Brightway!
